<img align="left" src = ./images/linea.png width=130 style="padding: 20px"> 

# PZ-Validation output with using RAIL

   **Contact**: Heloisa da S. Mengisztki, Julia Gschwend<br>
   **Last verified run**: 2024-june <br><br><br>

QA created to validate the quality of outputs produced by the `pz-compute` pipeline, documented in the [BRA-LIN in-kind contribution overview page](https://linea-it.github.io/pz-lsst-inkind-doc/s4_4/). The pipeline consists of a software layer to connect RAIL's user interface developed by LSST DESC to the Brazilian IDAC's HPC infrastructure (HPE Apollo Cluster and Lustre file system) and extract the maximum performance of it to allow the production of photo-z tables in large scale for LSST data releases. Details about RAIL are available on [RAIL's GitHub repository](https://github.com/LSSTDESC/RAIL) and on [Read The Docs](https://lsstdescrail.readthedocs.io/) page.   
  
This notebook is based on the notebooks present in the rail documentation "Goldenspike", "Evaluation_by_type" and "Evaluation_Demo", which can be found in the Rail repository available at [Rail-Github](https://github.com/LSSTDESC/rail/tree/main/examples/goldenspike_examples). Reference notebooks authors: RAIL team.


### Table of Contents
1. [Imports](#Imports)
2. [Reading the files](#Reading-the-files)
3. [Metrics Validation](#Metrics-Validation )
   - Sumarry Statistics
   - CDF based Metrics
4. [Plots](#Plots)
   - Redshift stacked distribution of the pdfs
   - Zphot vs. Ztrue
   - QQ Plot
   - KS Plot
5. [Summarized Metrics](#Summarized-Metrics)

-----

### Observations and Pre requisites

- Have a rail environment and kernel 

> Rail Install: https://rail-hub.readthedocs.io/en/latest/source/installation.html <br/>
> To install rail was needed to run the folowing command and in the apl cluster `CFLAGS="-O2 -std=gnu99" pip install pytdigest`
> In a short way, it changes the version for the default C used

- Have pz-server installed on the kernel

> this step can be ignored if you already have an output file and the ztrue file

- Have a validation set and the true z. You can use pz-server to dowload a validation set and ztrue files with the id `72_pzcompute_results_for_qa_validation`

> Util command to read the output file in the terminal if necessary
> `h5dump -H output1.hdf5`

- Copy file utils.py from [rail](https://github.com/LSSTDESC/rail/blob/main/examples/evaluation_examples/utils.py)

> to use the utils is required to install seaborn aswell

# Imports

In [ ]:
# Pre-requisites
import os
import numpy as np
import tables_io
import qp 
import os
import pandas as pd
import h5py

from matplotlib import pyplot as plt
from pathlib import Path

In [ ]:
# Rail modules
import rail

from rail.core.data import TableHandle, QPHandle
from rail.core.stage import RailStage
from rail.core.utils import find_rail_file

from rail.estimation.algos.naive_stack import NaiveStackSummarizer
from rail.estimation.algos.point_est_hist import PointEstHistSummarizer

from rail.evaluation.point_to_point_evaluator import PointToPointEvaluator # zmode true vs zmode calculado

from rail.evaluation.metrics.cdeloss import *

from ondemand_utils import plot_pit_qq, ks_plot #copied utils.py from rail/examples/evaluation_examples

from qp.metrics.pit import PIT
 

%matplotlib inline
%reload_ext autoreload
%autoreload 2

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

# PZ-server

In [ ]:
from pzserver import PzServer

In [ ]:
token = ""
pz_server = PzServer(token=token, host="pz-dev") 

In [ ]:
output_files = pz_server.download_product('72_pzcompute_results_for_qa_validation')

In [ ]:
! unzip <path-to-pz-compute-local-repo>/doc/72_pzcompute_results_for_qa_validation_402ab.zip

In [ ]:
! unzip <path-to-pz-compute-local-repo>/doc/validation_set.zip

# Reading the files

Here we are going to load the files and have a look trough the output file of a pz-estimate to better undestand the structure of the output data.

In [ ]:
validation_set_output_path = '<path-to-pz-compute-local-repo>/doc/validation_set_output.hdf5'
validation_set_path = '<path-to-pz-compute-local-repo>/doc/validation_set.hdf5'

In [ ]:
pdfs_file_output = find_rail_file(validation_set_output_path)
table = tables_io.read(pdfs_file_output, fmt='hdf5')
table

## Adding Zmode to the output

Adding the mode of the pdf generated for each object in the file, each zmode is equivalent to the photoz calculated.

In [ ]:
if 'ancil' in table:
    if 'zmode' in table['ancil']:
        print('file already has zmodes')
else:
    print('inserting zmodes')
    def zmode(df):
        results = []

        for index, row in df.iterrows():
            #mode_y = row.max()
            mode_x = row.idxmax()
            results.append(mode_x/100)

        return results

    zvalues_df = pd.DataFrame(table['data']['yvals'])

    z_modes = zmode(zvalues_df)

    def write_hdf5_file(file_name, z_modes):
        with h5py.File(file_name, 'r+') as hdf5_file:
            photometry_group = hdf5_file.create_group('ancil')
            photometry_group['zmode'] = z_modes

    write_hdf5_file(pdfs_file_output, z_modes)

In [ ]:
output_pdfs = DS.read_file(pdfs_file_output, QPHandle, pdfs_file_output)
output_pdfs().build_tables()

## Reading the Truth table

In [ ]:
ztrue_file = find_rail_file(validation_set_path)
ztrue_data = DS.read_file('ztrue_data', TableHandle, ztrue_file)

truth = tables_io.convertObj(ztrue_data(), tables_io.types.PD_DATAFRAME)
truth.head()

In [ ]:
print(f"ztrue com {len(truth)} objetos")
print(f"pdfs output com {len(output_pdfs().build_tables()['data']['yvals'])} objetos")

## Ploting some objects to see the pdfs

In [ ]:
x_vals = output_pdfs().metadata()['xvals'] #the photoz bins
y_vals = output_pdfs().build_tables()['data']['yvals'] #the pdfs

In [ ]:
zvalues_df = pd.DataFrame(y_vals)
print(f"Outputs com {len(zvalues_df)} objetos")
zvalues_df[:3]

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(3, hspace=0)
axs = gs.subplots(sharex=True, sharey=True)
fig.suptitle('pdf values for 3 objects')
axs[0].plot(x_vals[0], y_vals[1])
axs[1].plot(x_vals[0], y_vals[2])
axs[2].plot(x_vals[0], y_vals[3])

axs[2].set_xlabel("redshift")
axs[1].set_ylabel("fzboost pdfs")

for ax in axs:
    ax.label_outer()

# Metrics Validation 

Here we are going to calculate some metrics to validate the quality of the estimation.

## Point to point metrics - Sumary statistics

1. point_stats_iqr: 'Interquatile range from 0.25 to 0.75', i.e., the middle 50% of the distribution of point_stats_ez, robust to outliers
2. point_bias: Median of point_stats_ez
3. point_outlier_rate: Calculates the catastrophic outlier rate, defined in the Science Book as the number of galaxies with ez larger than max(0.06,3sigma). This keeps the fraction reasonable when sigma is very small.
4. point_stats_sigma_mad: Sigma of the median absolute deviation

In [ ]:
stage_dict = dict(
    metrics=['point_stats_ez', 'point_stats_iqr', 'point_bias', 'point_outlier_rate', 'point_stats_sigma_mad'],
    _random_state=None,
    hdf5_groupname='photometry',
    point_estimate_key='zmode',
    chunk_size=10000,
    metric_config={
        'point_stats_iqr':{'tdigest_compression': 100},
    }
)
ptp_stage = PointToPointEvaluator.make_stage(name='point_to_point', **stage_dict)

In [ ]:
ptp_results = ptp_stage.evaluate(output_pdfs, ztrue_data)
ptp_results

In [ ]:
results_df = tables_io.convertObj(ptp_results['summary'](), tables_io.types.PD_DATAFRAME)
results_df

## Dist to point metrics - CDF based Metrics

1. cdeloss: [Conditional Density Estimation](https://vitaliset.github.io/conditional-density-estimation/)
2. pit: [Probability Integral Transform](https://en.wikipedia.org/wiki/Probability_integral_transform)
3. cvm: [Cramer-von Mises](https://en.wikipedia.org/wiki/Cram%C3%A9r%E2%80%93von_Mises_criterion)
4. ks: [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test)
5. ad: [Anderson-Darling](https://en.wikipedia.org/wiki/Anderson%E2%80%93Darling_test)

### CDE Loss

In the absence of true photo-z posteriors, the metric used to evaluate individual PDFs is the **Conditional Density Estimate (CDE) Loss**, a metric analogue to the root-mean-squared-error:

$$ L(f, \hat{f}) \equiv  \int \int {\big(f(z | x) - \hat{f}(z | x) \big)}^{2} dzdP(x), $$ 

where $f(z | x)$ is the true photo-z PDF and $\hat{f}(z | x)$ is the estimated PDF in terms of the photometry $x$. Since $f(z | x)$  is unknown, we estimate the **CDE Loss** as described in [Izbicki & Lee, 2017 (arXiv:1704.08095)](https://arxiv.org/abs/1704.08095). :

$$ \mathrm{CDE} = \mathbb{E}\big(  \int{{\hat{f}(z | X)}^2 dz} \big) - 2{\mathbb{E}}_{X, Z}\big(\hat{f}(Z, X) \big) + K_{f},  $$


where the first term is the expectation value of photo-z posterior with respect to the marginal distribution of the covariates X, and the second term is the expectation value  with respect to the joint distribution of observables X and the space Z of all possible redshifts (in practice, the centroids of the PDF bins), and the third term is a constant depending on the true conditional densities $f(z | x)$. 

In [ ]:
cdelossobj = CDELoss(output_pdfs.data, output_pdfs().metadata()['xvals'].ravel(), ztrue_data()['redshift'])

cde_stat_and_pval = cdelossobj.evaluate()
print(cde_stat_and_pval)
print(f"CDE loss of this sample: {cde_stat_and_pval.statistic:.2f}") 

### PIT

The [Probability Integral Transform](https://en.wikipedia.org/wiki/Probability_integral_transform) (PIT), is the Cumulative Distribution Function (CDF) of the photo-z PDF 

$$ \mathrm{CDF}(f, q)\ =\ \int_{-\infty}^{q}\ f(z)\ dz $$

evaluated at the galaxy's true redshift for every galaxy $i$ in the catalog.

$$ \mathrm{PIT}(p_{i}(z);\ z_{i})\ =\ \int_{-\infty}^{z^{true}_{i}}\ p_{i}(z)\ dz $$ 

Probability integrated transform, 𝑃𝐼𝑇 (𝑧phot) = ∫𝑧phot 0 𝑃 (𝑧) 𝑑𝑧, calculated for all test galaxies, should be flat for well-estimated 𝑃 (𝑧) (Polsterer et al., 2016)

In [ ]:
pitobj = PIT(output_pdfs(), ztrue_data()['redshift'])

pit_out_rate = pitobj.evaluate_PIT_outlier_rate()
print(f"PIT outlier rate of this sample: {pit_out_rate:.6f}") 

### Kolmogorov-Smirnov  

Let's start with the traditional Kolmogorov-Smirnov (KS) statistic test, which is the maximum difference between the empirical and the expected cumulative distributions of PIT values:

$$
\mathrm{KS} \equiv \max_{PIT} \Big( \left| \ \mathrm{CDF} \small[ \hat{f}, z \small] - \mathrm{CDF} \small[ \tilde{f}, z \small] \  \right| \Big)
$$

Where $\hat{f}$ is the PIT distribution and $\tilde{f}$ is U(0,1). Therefore, the smaller value of KS the closer the PIT distribution is to be uniform. The `evaluate` method of the PITKS class returns a named tuple with the statistic and p-value. 

In [ ]:
ks_stat_and_pval = pitobj.evaluate_PIT_KS()
print(f"PIT KS stat and pval: {ks_stat_and_pval}") 

### Cramer-von Mises

Similarly, let's calculate the Cramer-von Mises (CvM) test, a variant of the KS statistic defined as the mean-square difference between the CDFs of an empirical PDF and the true PDFs:

$$ \mathrm{CvM}^2 \equiv \int_{-\infty}^{\infty} \Big( \mathrm{CDF} \small[ \hat{f}, z \small] \ - \ \mathrm{CDF} \small[ \tilde{f}, z \small] \Big)^{2} \mathrm{dCDF}(\tilde{f}, z) $$ 

on the distribution of PIT values, which should be uniform if the PDFs are perfect.

In [ ]:
cvm_stat_and_pval = pitobj.evaluate_PIT_CvM()
print(f"PIT CvM stat and pval: {cvm_stat_and_pval}")

### Anderson-Darling 

Another variation of the KS statistic is the Anderson-Darling (AD) test, a weighted mean-squared difference featuring enhanced sensitivity to discrepancies in the tails of the distribution. 

$$ \mathrm{AD}^2 \equiv N_{tot} \int_{-\infty}^{\infty} \frac{\big( \mathrm{CDF} \small[ \hat{f}, z \small] \ - \ \mathrm{CDF} \small[ \tilde{f}, z \small] \big)^{2}}{\mathrm{CDF} \small[ \tilde{f}, z \small] \big( 1 \ - \ \mathrm{CDF} \small[ \tilde{f}, z \small] \big)}\mathrm{dCDF}(\tilde{f}, z) $$ 



In [ ]:
ad_stat_crit_sig = pitobj.evaluate_PIT_anderson_ksamp()
print(f"PIT AD stat and pval: {ad_stat_crit_sig}")

It is possible to remove catastrophic outliers before calculating the integral for the sake of preserving numerical instability. For instance, Schmidt et al. computed the Anderson-Darling statistic within the interval (0.01, 0.99).

In [ ]:
ad_stat_crit_sig_cut = pitobj.evaluate_PIT_anderson_ksamp(pit_min=0.01, pit_max=0.99)
print(f"AD metric for 0.01 < PIT < 0.99 and pval: {ad_stat_crit_sig_cut}") 

# Plots

### Redshift stacked distribution of the pdfs
Summarized the per-galaxy redshift constraints to make population-level distributions

In [ ]:
point_estimate_test = PointEstHistSummarizer.make_stage(name="point_estimate_test")
naive_stack_test = NaiveStackSummarizer.make_stage(name="naive_stack_test")

point_estimate_ens = point_estimate_test.summarize(output_pdfs)
naive_stack_ens = naive_stack_test.summarize(output_pdfs)

In [ ]:
_ = naive_stack_ens.data.plot_native(xlim=(0, 3)) #pdfs

In [ ]:
_ = point_estimate_ens.data.plot_native(xlim=(0, 3)) #zmode

### Zphot vs. Ztrue

This plot is a scatter plot that compares two types of redshift measurements: photometric redshift estimated ("photoz") and spectroscopic redshift ("specz") representing the true redshift

The red dashed line is the 1:1 line where photoz would equal specz. This line represents perfect agreement between the two measurements. Therefore the scatter around the red line suggests that, on average, the photoz estimates are reasonably accurate but with some variation. Less scatter indicates better agreement. Systematic deviations or patterns can also appear compared to the red line when indicating systematic biases or errors in the photometric redshift estimates.

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(ztrue_data.data['redshift'],output_pdfs.data.ancil['zmode'],s=1,c='k',label='simple fzboost mode')
plt.plot([0,3],[0,3],'r--')
plt.xlim([0,3])
plt.ylim([0,3])
plt.xlabel("specz")
plt.ylabel("photoz")

### QQ Plot

The histogram of PIT values is a useful tool for a qualitative assessment of PDFs quality. It shows whether the PDFs are:
* biased (tilted PIT histogram)
* under-dispersed (excess counts close to the boudaries 0 and 1)
* over-dispersed (lack of counts close the boudaries 0 and 1)
* well-calibrated (flat histogram)

Following the standards in DC1 paper, the PIT histogram is accompanied by the quantile-quantile (QQ), which can be used to compare qualitatively the PIT distribution obtained with the PDFs agaist the ideal case (uniform distribution). The closer the QQ plot is to the diagonal, the better is the PDFs calibration. 

The black horizontal line represents the ideal case where the PIT histogram would behave as a uniform distribution U(0,1).

In [ ]:
plot_pit_qq(output_pdfs.data.objdata()['yvals'], output_pdfs().metadata()['xvals'].ravel(), ztrue_data()['redshift'], title="PIT-QQ plot", code="FlexZBoost", pit_out_rate=pit_out_rate, savefig=False)

### KS plot

The two lines represent the CDFs of two distributions:

- The red line ("uniform") represents the CDF of a uniform distribution, which is the reference or ideal model.
- The blue line ("sample PIT") represents the CDF of the model predictions or the Probability Integral Transform (PIT) values.

In this graph max indicates the maximum difference between the two CDFs. It is a measure of the discrepancy between the predicted distribution and the ideal uniform distribution. In the graph, this difference is highlighted by the black arrow labeled.

The KS value on the graph quantifies the distance between the two cumulative distributions. A smaller KS value indicates that the two distributions are closer to each other.

In [ ]:
ks_plot(pitobj)

# Summarized Metrics 


RAIL evaluate metrics that guarantee quality of the processed data, its subdivided in PIT metrics, based on the CDF of the generated photozs described in (Schmidt et al., 2020), CDE metrics, based on the cumulative density function of the MISSING WORDS HELP and the point metrics statistic better disposed in the table:

| metric            | classification |limits | reference |
| :---------------- | :----:|:----: | :-------: |
| STD DEV     | POINT Metric | < 0.05(1 + zphot) |  [Schmidt et al., 2020](http://doi.org/10.1093/mnras/staa2799)| 
| BIAS              | POINT Metric | < 0.003    | [Schmidt et al., 2020](http://doi.org/10.1093/mnras/staa2799)          | 
| OUTRATE | POINT Metric | < 10% | [Schmidt et al., 2020](http://doi.org/10.1093/mnras/staa2799)        | 
| CDE loss          | CDE metric | lower the better | [Izbicki & Lee, 2017](https://arxiv.org/abs/1704.08095) |
| PIT               | PIT Metric |prox de 1 | [Polsterer et al., 2016](https://arxiv.org/abs/1608.08016) |
| AD | PIT Metrics | lower indicates a uniform PIT | [Schmidt et al., 2020](http://doi.org/10.1093/mnras/staa2799) |
| CVM | PIT Metrics | lower indicates a uniform PIT | [Schmidt et al., 2020](http://doi.org/10.1093/mnras/staa2799) |
| KS | PIT Metrics | lower indicates a uniform PIT | [Schmidt et al., 2020](http://doi.org/10.1093/mnras/staa2799) |



Note that this preliminary metrics proposed performance has been shown to be achievable for simulated LSST data with existing photo-𝑧 estimators (e.g., Graham et al., 2018;
Schmidt et al., 2020).

Based on the science use-cases described in Appendix C of the [DMTN-049](https://dmtn-049.lsst.io/), the Object catalog photo-zs could have a point-estimate accuracy of 10 % and still meet the basic science needs. The photo-zs results should result in a standard deviation of 𝑧true − 𝑧phot of 𝜎𝑧 < 0.05(1 + 𝑧phot) , and a catastrophic outlier fraction of 𝑓outlier < 10%, over a redshift range of 0.0 < 𝑧phot < 2.0 for galaxies with 𝑖 < 25 mag galaxies. 

In [ ]:
print("STD DEV: ", results_df['point_stats_iqr'][0])
print("BIAS: ", results_df['point_bias'][0])
print("OUTRATE: ", results_df['point_outlier_rate'][0])
print("PIT: ", pit_out_rate)
print("CDE loss: ", cde_stat_and_pval.statistic, " p-value: ", cde_stat_and_pval.p_value)
print("AD: ", ad_stat_crit_sig.statistic, " p-value: ", ad_stat_crit_sig.pvalue)
print("CVM: ", cvm_stat_and_pval.statistic, " p-value: ", cvm_stat_and_pval.pvalue)
print("KS: ", ks_stat_and_pval.statistic, " p-value: ", ks_stat_and_pval.pvalue)